In [9]:
import pandas as pd
import scipy.stats as stats

In [38]:
import seaborn as sns

# Cargar el dataset de Titanic
df = sns.load_dataset('titanic')

In [39]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [75]:
def get_features_cat_regression(df:pd.DataFrame, target_col:str, pvalue:float=0.05):
    """
    Identifica columnas categóricas en un DataFrame que están significativamente relacionadas con
    una columna objetivo numérica, utilizando el análisis de varianza (ANOVA).

    Argumentos:
    df (pd.DataFrame): DataFrame que contiene los datos a analizar.
    target_col (str): Nombre de la columna objetivo que debe ser numérica.
    pvalue (float, opcional): Umbral de p-valor para considerar significativas las columnas categóricas.

    Retorna:
    List[str]: Lista de nombres de columnas categóricas que tienen una relación estadísticamente significativa
               con la columna objetivo, basada en un p-valor menor o igual al umbral especificado.
    """
    # Verificar que p-valor es una probabilidad
    if not 0 < pvalue < 1:
        print("Error: El p-valor debe estar en [0,1].")
        return None
    # Verificar que target_col está en el DataFrame
    if target_col not in df.columns:
        print(f"Error: La columna '{target_col}' no se encuentra en el DataFrame.")
        return None

    # Verificar que target_col es numérica
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print(f"Error: La columna '{target_col}' debe ser numérica.")
        return None
    cardinalidad = df[target_col].nunique()
    total_filas = len(df)
    porcentaje_cardinalidad = (cardinalidad / total_filas) * 100

    # Verificar que target_col tiene cardinalidad suficiente para ser continua o discreta
    if porcentaje_cardinalidad<=10:
        print(f"Error: La columna '{target_col}' no tiene suficiente cardinalidad para ser considerada númerica continua o discreta.")
        return None

    
    # Lista para guardar las columnas categóricas significativas
    significant_cats = []
    for col in df.select_dtypes(include=['object', 'category']).columns:
        grouped = df.groupby(col, observed=True)[target_col].apply(list)
        _, p_val = stats.f_oneway(*grouped)
        if p_val <= pvalue:
            significant_cats.append(col)

    return significant_cats

In [76]:
get_features_cat_regression(df,'fare')

['sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive']